# Generate land/water training data on Sentinel-1 data using Sentinel-2 data

## Load packages

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import datacube
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points

from deafrica_tools.datahandling import load_ard, mostcommon_crs
from deafrica_tools.bandindices import calculate_indices
from dea_tools.coastal import model_tides, tidal_tag, pixel_tides, tidal_stats
from dea_tools.spatial import subpixel_contours
from deafrica_tools.plotting import display_map, rgb, map_shapefile
from deafrica_tools.dask import create_local_dask_cluster
from coastlines.raster import tide_cutoffs,load_tidal_subset
from coastlines.vector import points_on_line, annual_movements, calculate_regressions

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage.filters import threshold_minimum, threshold_otsu
from skimage.morphology import binary_dilation,disk
import random
import pickle

from modules import lee_filter,filter_by_tide_height,query_filter_s1,preprocess_s1

## Set up a Dask cluster

In [2]:
client = create_local_dask_cluster(return_client=True)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 26.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43015,Workers: 1
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 26.21 GiB
Comm: tcp://127.0.0.1:40761,Total threads: 4
Dashboard: /user/whusggliuqx@gmail.com/proxy/37869/status,Memory: 26.21 GiB
Nanny: tcp://127.0.0.1:46277,


## Connect to the datacube

In [3]:
dc = datacube.Datacube(app="Sentinel-1 training data collection")

## Define parameters

In [4]:
# Define areas of interest: Madagarscar,Comoros
locations={'Madagarscar_west':(-17.474,43.924),
           'Madagarscar_south':(-25.572,45.538),
           'Tanzania':(-6.32,39.280),
           'Kenya':(-4.025,39.725),
           'Comoros':(-12.4,43.736)
          }

# Set the range of dates for the analysis, time step and tide range
time_range = ('2018', '2021')
time_step = '1Y'
tide_range = (0.25, 0.75)

# whether to implement Lee filtering on Sentinel-1 data
lee_filtering=False

# Lee filtering size
filter_size=2

# percentage cloud-free pixels for Sentinel-2
clear_frac=0.9

out_folder='data'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

## Collect training data for all locations

In [5]:
def collect_training_samples(S2_filtered,ds_summaries_s1):
    # median of S2
    ds_summaries_s2 = (S2_filtered[['MNDWI']]
                    .resample(time=time_step)
                    .median('time')
                    .compute()
                        )
    # calculate water and land frequency
    ds_summaries_s2['water_freq']=(S2_filtered['MNDWI']>=0).resample(time=time_step).mean('time')
    ds_summaries_s2['land_freq']=(S2_filtered['MNDWI']<0).resample(time=time_step).mean('time')

    print('Collecting water/non-water samples...')
    # classs: water (1) and land (0)
    # simplified implementation of limiting samples around the coastal zone: excluding pixels that are almost always water/land
    ds_summaries_s2['water']=xr.where((ds_summaries_s2['MNDWI']>=0)&(ds_summaries_s2['water_freq']<0.8),1,
                                      xr.where((ds_summaries_s2['MNDWI']<0)&(ds_summaries_s2['land_freq']<0.8),0,
                                               np.nan))
    ds_summaries_s1=ds_summaries_s1.where(~ds_summaries_s2['water'].isnull(),np.nan)
    # reshape arrays
    s1_data=ds_summaries_s1.to_array(dim='variable').transpose('x','y','time', 'variable').values
    data_shape = s1_data.shape
    data=s1_data.reshape(data_shape[0]*data_shape[1]*data_shape[2],data_shape[3])
    labels=ds_summaries_s2.water.transpose('x','y','time').values.reshape(data_shape[0]*data_shape[1]*data_shape[2],)
    
    # remove NaNs
    labels=labels[~np.isnan(data).any(axis=1)]
    data=data[~np.isnan(data).any(axis=1)]
    print('Number of samples available: ',data.shape[0])
    
    # random sampling max 10000 samples per location
    if data.shape[0]>10000:
        rand_indices=random.sample(range(0, data.shape[0]), 10000)
        labels=labels[rand_indices]
        data=data[rand_indices]
        
    return data,labels

In [6]:
for location_name in locations:
    
    # define area of interest
    lat=locations[location_name][0]
    lon=locations[location_name][1]
    # Combine central lat,lon with buffer to get area of interest
    buffer = 0.04
    lat_range = (lat - buffer, lat + buffer)
    lon_range = (lon - buffer, lon + buffer)
    
    # query S1 data
    print('querying S1 data...')
    query = {
        'x': lon_range,
        'y': lat_range,
        'time': time_range,
        'measurements': ['vh','vv','mask','area'], # loading vh for experiement
        'resolution': (-20, 20),
    }
    # Identify the most common projection system in the input query
    output_crs = "EPSG:6933" #mostcommon_crs(dc=dc, product='s2_l2a', query=query)
    # filter by orbit frequency
    S1=query_filter_s1(dc,query,output_crs)

    # query S2
    print('querying S2 vdata...')
    query_s2=query.copy()
    query_s2.update({'measurements': ['red', 'green', 'blue', 'swir_1','nir']})
    S2 = load_ard(dc=dc,
                  products=['s2_l2a'],
                  output_crs=output_crs,
                  resampling='bilinear',
                  min_gooddata=0.6,
                  align=(10, 10),
                  mask_filters=[("opening", 2), ("dilation", 5)],
                  dask_chunks={'time': 1},
                  group_by='solar_day',
                  **query_s2)
    
    # Calculate S2 water index
    S2 = calculate_indices(S2, index='MNDWI', satellite_mission='s2')
    
    # Filter Sentinel-2 images by percentage clear pixels
    S2=S2.where(S2.MNDWI.isnull().mean(dim=['x','y'])<1-clear_frac,drop=True)

    # per-pixel tide modelling and filtering
    print('Tide modelling and filtering for S1...')
    S1_filtered=filter_by_tide_height(S1,tide_centre=0.0)
    S2_filtered=filter_by_tide_height(S2,tide_centre=0.0)

    # preprocess S1
    S1_filtered=preprocess_s1(S1_filtered,lee_filtering,filter_size,time_step)
    
    # save sentinel-1 feature names
    feature_names=list(ds_summaries_s1.keys())
    with open('s1_features', 'wb') as outfile:
        pickle.dump(feature_names, outfile)
    
    # generate training samples using S2
    data,labels=collect_training_samples(S2_filtered,ds_summaries_s1)
    
    # save training data
    print('Saving data...')
    out_file=os.path.join(out_folder,location_name)
    if not os.path.exists(out_file):
        np.savez(out_file,data=data,labels=labels)
        

querying S1 data...
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Returning 1 time steps as a dask array
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Returning 145 time steps as a dask array


/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


querying S2 vdata...
Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a


/home/jovyan/.local/lib/python3.10/site-packages/deafrica_tools/datahandling.py:238: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn(


Counting good quality pixels for each time step
Filtering to 220 out of 288 time steps with at least 60.0% good quality pixels
Applying morphological filters to pq mask [('opening', 2), ('dilation', 5)]
Applying pixel quality/cloud mask
Returning 220 time steps as a dask array


KeyError: 'Indexing with a boolean dask array is not allowed. This will result in a dask array of unknown shape. Such arrays are unsupported by Xarray.Please compute the indexer first using .compute()'

In [ ]:
# Shut down Dask client now that we have processed the data we need
client.close()